In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1UXqC5sUIkvO5TdPP9iZxFPlmI_lOkL21/view?usp=sharing'

In [ ]:
fluff, id = link.split('=')
print (id)

sharing


In [ ]:
downloaded = drive.CreateFile({'id':'1UXqC5sUIkvO5TdPP9iZxFPlmI_lOkL21'})

In [ ]:
downloaded.GetContentFile('final_11-13-20.csv') 

In [ ]:
import pandas as pd
df = pd.read_csv('final_11-13-20.csv')

In [ ]:
df1 = df[['status_id','text']]

In [1]:
import pandas as pd 
import numpy as np
import re
import collections
import matplotlib.pyplot as plt
from IPython.display import Image

# Packages for text processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,LancasterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
from google.colab import files
uploaded = files.upload()

Saving eva.xlsx to eva.xlsx


In [ ]:
data = pd.read_excel('eva.xlsx')
data.drop(['id','Column4'],axis=1,inplace=True)
data.head(5)

In [5]:
data['labels'][data['labels'] == 'neutral'] = 1
data['labels'][data['labels'] == 'positive'] = 2
data['labels'][data['labels'] == 'negative'] = 0
data.dropna(axis=0,how='any',inplace=True)

In [19]:
stop_words = set(stopwords.words('english'))

In [20]:
def normalizer(tweet): 
    # tokenize
    tokens = nltk.word_tokenize(tweet) 
    # convert everything to lowercase
    lower_case = [l.lower() for l in tokens] 
    # remove words in stop_words and keep words in whitelist
    filtered_result = list(filter(lambda l: l not in stop_words and l not in ['@','-'], lower_case)) 
    normalized_tweet = " ".join(filtered_result) 
    return normalized_tweet

In [ ]:
normalizer(df1['text'][2])

"facts : chernobyl n't prove nuclear energy dangerous — shows failures communist soviet union ! accident turned catastrophe pitiful safety procedures . even , initial reports radiation turned grossly exaggerated ."

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
data['context'] = data['context'].apply(normalizer)

In [6]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 5.2MB/s 


In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [22]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))

In [ ]:
sentiment_analyzer_scores(df['text'][0])

stockfttp ellektrikk nope as nuclear engineers we study every different type of energy their damage to the environment and it s almost laughable that we get replaced by what people think is grean but of course we understand the dangers of nuclear more than anyone else as well {'neg': 0.136, 'neu': 0.787, 'pos': 0.077, 'compound': -0.5017}


In [ ]:
sentiment_analyzer_scores('@#@This bad')

@#@This bad----------------------------- {'neg': 0.778, 'neu': 0.222, 'pos': 0.0, 'compound': -0.5423}


In [ ]:
analyser.polarity_scores('This \n\rr is bad!')['compound']

-0.5848

In [23]:
def get_score(tweet):
  score = analyser.polarity_scores(tweet)['compound']
  if score >= 0.05:
    senti = 2
  elif score <= -0.05:
    senti = 0
  else:
    senti = 1
  return senti

In [24]:
data['score'] = data.context.apply(get_score)

In [26]:
data.score.value_counts()

2    9946
1    5629
0    5057
Name: score, dtype: int64

In [27]:
sum(data['score']==data['labels'])/len(data)

0.5133772780147344